# QLearning example

## Scala dependencies

In [1]:
import $ivy.`io.github.cric96::scalapy-gym:0.1.0`

import $ivy.$                                    

## Environment definition
The agent controls the movement of a character in a grid world. Some tiles of the grid are walkable, and others lead to the agent falling into the water. Additionally, the movement direction of the agent is uncertain and only partially depends on the chosen direction. The agent is rewarded for finding a walkable path to a goal tile.

In [2]:
import io.github.cric96.gym.envs.EnvFactory.ToyText
import io.github.cric96.gym.core.Env.StepResponse
import scala.util.Random
import me.shadaj.scalapy.py
type Reward = Double
val env = ToyText.frozenLakeV1()
val actionSpace = env.actionSpace
val observationSpace = env.observationSpace
val random = new Random(42)

import io.github.cric96.gym.envs.EnvFactory.ToyText

import io.github.cric96.gym.core.Env.StepResponse

import scala.util.Random

import me.shadaj.scalapy.py

defined type Reward
env: io.github.cric96.gym.core.Env[Int, Int, io.github.cric96.gym.spaces.Discrete, io.github.cric96.gym.spaces.Discrete] = <TimeLimit<FrozenLakeEnv<FrozenLake-v1>>>
actionSpace: io.github.cric96.gym.spaces.Discrete[Int] = Discrete(4)
observationSpace: io.github.cric96.gym.spaces.Discrete[Int] = Discrete(16)
random: Random = scala.util.Random@3eb9f7e7

## Random exploration

In [3]:
def step = {
  env.step(actionSpace.sample())
}
val maxIteration = 1000

def simulateFor(iter: Int): List[StepResponse[Int]] = {
  val value = env.step(actionSpace.sample())
  (iter, value.done) match {
    case (n, true) => List(value)
    case (n, _) if n > 0 => value :: simulateFor(n - 1)
    case _ => Nil
  }
}
env.reset()
val track = simulateFor(maxIteration)

println(s"time necessary: ${track.length}")
println(s"total reward: ${track.map(_.reward).sum}")

time necessary: 6
total reward: 0.0


defined function step
maxIteration: Int = 1000
defined function simulateFor
res2_3: Int = 0
track: List[StepResponse[Int]] = List(
  StepResponse(0, 0.0F, false, {'prob': 0.3333333333333333}),
  StepResponse(0, 0.0F, false, {'prob': 0.3333333333333333}),
  StepResponse(0, 0.0F, false, {'prob': 0.3333333333333333}),
  StepResponse(4, 0.0F, false, {'prob': 0.3333333333333333}),
  StepResponse(4, 0.0F, false, {'prob': 0.3333333333333333}),
  StepResponse(5, 0.0F, true, {'prob': 0.3333333333333333})
)

## Average random search

In [4]:
val howManySearch = 1000
val searches = (0 to howManySearch).to(LazyList)
  .tapEach(i => env.reset())
  .map(i => simulateFor(maxIteration))
  .map(_.length)
  .sum

val averageStep = searches / howManySearch.toDouble

howManySearch: Int = 1000
searches: Int = 7798
averageStep: Double = 7.798

## QTable

In [9]:
import io.github.cric96.gym.spaces.Discrete
type QTable = Map[(Int, Int), Reward]
val baseTable: QTable = (for {
  i <- 0 until actionSpace.n
  j <- 0 until observationSpace.n
} yield ((i, j) -> random.nextDouble)).toMap

import io.github.cric96.gym.spaces.Discrete

defined type QTable
baseTable: QTable = HashMap(
  (2, 6) -> 0.2967796276363246,
  (3, 15) -> 0.10620241350039672,
  (1, 8) -> 0.80498077524871,
  (1, 4) -> 0.7887192251513687,
  (3, 2) -> 0.799564698602399,
  (0, 12) -> 0.2773936756099694,
  (0, 2) -> 0.147033388214162,
  (2, 9) -> 0.28074577309289495,
  (2, 2) -> 0.5463416851906217,
  (3, 4) -> 0.8478799731826525,
  (2, 1) -> 0.030664345240818958,
  (3, 13) -> 0.08085989702518803,
  (2, 5) -> 0.01814815040344353,
  (2, 14) -> 0.04353198427290006,
  (2, 0) -> 0.6876641637096393,
  (1, 13) -> 0.981775856893166,
  (0, 15) -> 0.6903356213393419,
  (1, 2) -> 0.07560994051627679,
  (2, 12) -> 0.24761162636194045,
  (0, 1) -> 0.32443120588354113,
  (0, 14) -> 0.19673520090081031,
  (1, 1) -> 0.7539480637337175,
  (0, 4) -> 0.9175065627648522,
  (2, 13) -> 0.728590142479296,
  (2, 7) -> 0.5723809221850318,
  (3, 14) -> 0.9044178250003002,
  (3, 11) -> 0.4124771572093511,
  (1, 9) -> 0.097627968783

## QLearning algorithm

In [6]:
type State = Int

def spin(proba: Double): Boolean = random.nextDouble < proba

def maxByObs(qTable: QTable, obs: Int): (State, Reward) = qTable
      .filter { case ((action, obs), reward) => obs == obs }
      .map { case ((action, obs), reward) => action -> reward }
      .maxBy { case (action, reward) => reward }


def qLearningStep(qTable: QTable,
              prevObs: State,
              discount: Double, 
              learningRate: Double, 
              epsilon: Double): (QTable, StepResponse[Int]) = { // return an improved table
  val action = if(spin(epsilon)) {
    actionSpace.sample()
  } else {
    maxByObs(qTable, prevObs)._1
  }
  val stepResponse = env.step(action)
  val currentValue = qTable((action, prevObs))
  val optimalFutureValue = maxByObs(qTable, stepResponse.observation)._2
  val temporalDifference = stepResponse.reward + discount * optimalFutureValue - currentValue
  val qTableUpdate = currentValue + learningRate * temporalDifference
  val newTable = qTable + ((action, prevObs) -> qTableUpdate)
  (newTable, stepResponse)
}

def qLearning(qTable: QTable,
              maxIter: Int = 1000,
              discount: Double = 0.9, 
              learningRate: Double = 0.01,
              epsilon: Double = 0.1): QTable = {
  def loop(iter: Int, data: (QTable, StepResponse[Int])): QTable = (iter, data) match {
    case (iter, (qTable, _)) if (iter <= 0) => qTable
    case (iter, (qTable, step)) if(step.done) => qTable
    case (iter, (qTable, step)) => 
      val nextData = qLearningStep(qTable, step.observation, discount, learningRate, epsilon)
      loop(iter - 1, nextData)
  }
  val initialState = env.reset()
  val initialStep = StepResponse(initialState, 0.0f, false, py.Dynamic.global.None)
  loop(maxIter, (qTable, initialStep))
}

defined type State
defined function spin
defined function maxByObs
defined function qLearningStep
defined function qLearning

## Program execution

In [16]:
val learningEpisodes = 500
val qStarTable = (0 to learningEpisodes)
  .to(LazyList)
  .tapEach(i => println(s"Iter $i"))
  .foldLeft(baseTable){ case (table, _) => qLearning(table)}

Iter 0
Iter 1
Iter 2
Iter 3
Iter 4
Iter 5
Iter 6
Iter 7
Iter 8
Iter 9
Iter 10
Iter 11
Iter 12
Iter 13
Iter 14
Iter 15
Iter 16
Iter 17
Iter 18
Iter 19
Iter 20
Iter 21
Iter 22
Iter 23
Iter 24
Iter 25
Iter 26
Iter 27
Iter 28
Iter 29
Iter 30
Iter 31
Iter 32
Iter 33
Iter 34
Iter 35
Iter 36
Iter 37
Iter 38
Iter 39
Iter 40
Iter 41
Iter 42
Iter 43
Iter 44
Iter 45
Iter 46
Iter 47
Iter 48
Iter 49
Iter 50
Iter 51
Iter 52
Iter 53
Iter 54
Iter 55
Iter 56
Iter 57
Iter 58
Iter 59
Iter 60
Iter 61
Iter 62
Iter 63
Iter 64
Iter 65
Iter 66
Iter 67
Iter 68
Iter 69
Iter 70
Iter 71
Iter 72
Iter 73
Iter 74
Iter 75
Iter 76
Iter 77
Iter 78
Iter 79
Iter 80
Iter 81
Iter 82
Iter 83
Iter 84
Iter 85
Iter 86
Iter 87
Iter 88
Iter 89
Iter 90
Iter 91
Iter 92
Iter 93
Iter 94
Iter 95
Iter 96
Iter 97
Iter 98
Iter 99
Iter 100
Iter 101
Iter 102
Iter 103
Iter 104
Iter 105
Iter 106
Iter 107
Iter 108
Iter 109
Iter 110
Iter 111
Iter 112
Iter 113
Iter 114
Iter 115
Iter 116
Iter 117
Iter 118
Iter 119
Iter 120
Iter 121
Iter 122
Ite

learningEpisodes: Int = 500
qStarTable: QTable = HashMap(
  (2, 6) -> 0.3091398637261846,
  (3, 15) -> 0.10620241350039672,
  (1, 8) -> 0.7185098912195833,
  (1, 4) -> 0.7139313812359247,
  (3, 2) -> 0.7961511499563005,
  (0, 12) -> 0.2773936756099694,
  (0, 2) -> 0.15269240614182025,
  (2, 9) -> 0.2935822306965494,
  (2, 2) -> 0.5560907116749804,
  (3, 4) -> 0.8301679041610323,
  (2, 1) -> 0.09590166683620681,
  (3, 13) -> 0.0934381951758885,
  (2, 5) -> 0.01814815040344353,
  (2, 14) -> 0.04353198427290006,
  (2, 0) -> 0.694053382080211,
  (1, 13) -> 0.8344629014598858,
  (0, 15) -> 0.6903356213393419,
  (1, 2) -> 0.5827019350853763,
  (2, 12) -> 0.24761162636194045,
  (0, 1) -> 0.3397398126332017,
  (0, 14) -> 0.2070075805043859,
  (1, 1) -> 0.7139609196150255,
  (0, 4) -> 0.8942641589801559,
  (2, 13) -> 0.7284335928643029,
  (2, 7) -> 0.5723809221850318,
  (3, 14) -> 0.9025029985600971,
  (3, 11) -> 0.4124771572093511,
  (1, 9) -> 0.5133015064901915,
  (1, 6) -> 0.5605440822074867

## Simulation using Q*Table

In [18]:
def simulateWithQTable(iter: Int, qTable: QTable, prevObs: Int): List[StepResponse[Int]] = {
  val action = maxByObs(qTable, prevObs)._1
  val value = env.step(action)
  (iter, value.done) match {
    case (n, true) => List(value)
    case (n, _) if n > 0 => value :: simulateWithQTable(n - 1, qTable, value.observation)
    case _ => Nil
  }
}

val howManySearch = 1000
val searches = (0 to howManySearch).to(LazyList)
  .map(i => env.reset())
  .map(obs => simulateWithQTable(maxIteration, qStarTable, obs))
  .map(_.length)
  .sum

val averageStep = searches / howManySearch.toDouble

defined function simulateWithQTable
howManySearch: Int = 1000
searches: Int = 5067
averageStep: Double = 5.067